# Importing The Model Building Libraries

In [57]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten

In [58]:
from tensorflow.keras.models import Model

In [59]:
from tensorflow.keras.applications.vgg16 import VGG16

In [60]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [61]:
from tensorflow.keras.preprocessing import image

In [62]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [63]:
from tensorflow.keras.models import Sequential

In [64]:
import numpy as np

In [65]:
from glob import glob

# Image Data Agumentation

In [85]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Loading Our Data And Performing Data Agumentation

In [98]:
train_set = train_datagen.flow_from_directory('../dataset/train_set',
                                                 target_size = (244, 244),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 316 images belonging to 3 classes.


In [99]:
test_set = test_datagen.flow_from_directory('../dataset/test_set',
                                            target_size = (244, 244),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 120 images belonging to 3 classes.


In [102]:
print(train_set.class_indices)#checking the number of classes

{'crack': 0, 'flakes': 1, 'roof': 2}


In [103]:
from collections import Counter as c
c(train_set .labels)

Counter({0: 86, 1: 176, 2: 54})

# Resizing The Images

In [66]:
IMAGE_SIZE=[244,244]

In [67]:
train_path='dataset\train_set'

In [68]:
valid_path='dataset\test_set'

# Pre-Trained CNN Model As A Feature Extractor

In [69]:
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [70]:
for layer in vgg16.layers:
    layer.trainable=False

In [71]:
folders=glob('../dataset/train_set/*')

In [72]:
folders

['../dataset/train_set\\crack',
 '../dataset/train_set\\flakes',
 '../dataset/train_set\\roof']

In [73]:
x=Flatten()(vgg16.output)

# Adding Dense Layers

In [74]:
prediction=Dense(len(folders),activation='softmax')(x)

In [75]:
model=Model(inputs=vgg16.input,outputs=prediction)

In [76]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 244, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 244, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 122, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 122, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 122, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 61, 61, 128)       0   

# Configure The Learning Process

In [77]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train The Model

In [123]:
r=model.fit_generator(train_set,validation_data=test_set,epochs=10,steps_per_epoch=len(train_set),validation_steps=len(test_set))

Epoch 1/10
10/10 [==============================] - 239s 24s/step - loss: 0.0940 - acc: 0.9810 - val_loss: 0.5811 - val_acc: 0.8000
Epoch 2/10
10/10 [==============================] - 231s 23s/step - loss: 0.0890 - acc: 0.9905 - val_loss: 0.5182 - val_acc: 0.8000
Epoch 3/10
10/10 [==============================] - 231s 23s/step - loss: 0.0733 - acc: 0.9905 - val_loss: 0.6488 - val_acc: 0.8000
Epoch 4/10
10/10 [==============================] - 232s 23s/step - loss: 0.0691 - acc: 0.9873 - val_loss: 0.5865 - val_acc: 0.8083
Epoch 5/10
10/10 [==============================] - 235s 23s/step - loss: 0.0659 - acc: 0.9873 - val_loss: 0.5682 - val_acc: 0.8000
Epoch 6/10
10/10 [==============================] - 199s 20s/step - loss: 0.0610 - acc: 0.9905 - val_loss: 0.6405 - val_acc: 0.8000
Epoch 7/10
10/10 [==============================] - 188s 19s/step - loss: 0.0537 - acc: 0.9937 - val_loss: 0.6048 - val_acc: 0.8083
Epoch 8/10
10/10 [==============================] - 191s 19s/step - loss: 0.

# Save The Model

In [126]:
from tensorflow.keras.models import load_model
model.save('model_building_defects_vgg16.h5')

# Test The Model

In [127]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model=load_model("model_building_defects_vgg16.h5")

In [136]:
img=image.load_img(r'D:\dataset\test_set\flakes\38.jpg',target_size=(244,244))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
a=np.argmax(model.predict(img_data), axis=1)

In [137]:
a

array([1], dtype=int64)

In [138]:
index=['crack','flakes','roof']
result=str(index[a[0]])
result

'flakes'